In [184]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K

n_features = 4
n_runs = 300
seq_length = [180, 360, 540, 720, 900]
labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_len_labels.csv")
labels = labels.drop(columns="Unnamed: 0") 
labels = np.array(labels)

def read_sequences():
    run_list_mix = []
    run_list_ordered = []
    labels_reordered = []
    for index in range(n_runs):
        run_csv = pd.read_csv("Mix_sequences_var_length/run^"+str(index))
        run_csv = run_csv.drop(columns=['Unnamed: 0'])
        run_list_mix.append(run_csv)
    for index, length in enumerate(seq_length):
        run_list_ordered.append([])
        for run_index,run in enumerate(run_list_mix): 
            if len(run) == length:
                run_list_ordered[index].append(run)    
                labels_reordered.append(labels[run_index])
    return run_list_ordered, labels
    
    
    
def stadard_sequences(sequences):
    for index, seqs in enumerate(sequences):
        for i, seq in enumerate(seqs):
            seqs[i] = MinMaxScaler(feature_range=[0,1]).fit_transform(seq)
    return sequences                    


trainX_sequences, labels_ordered = read_sequences()
trainX_sequences = stadard_sequences(trainX_sequences)
for i, leng in enumerate(seq_length):
    trainX_sequences[i] = np.reshape(trainX_sequences[i], (-1, leng, n_features))

# def train_generator():
#     index = 0
#     while True:
#         x_train = trainX_sequences[index]
#         index += 1
#         yield (x_train, x_train) 


In [191]:
from keras import Sequential
from keras.layers import Conv2DTranspose, LSTM, RepeatVector
from keras.losses import mse
import tensorflow as tf

intermediate_dimensions = 50
latent_dim = 20
filters = 50


def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def reshape(x):
    x_shape = K.shape(x)
    new_x_shape = K.concatenate([x_shape[:-2], [x_shape[-2] * x_shape[-1]]])    
    print(new_x_shape)
    return K.reshape(x, new_x_shape)

def create_ae(verbose):
    filters = 50
    input = Input(shape=(None, n_features))
    x = input
    
    for i in range(2):
        x = Conv1D(filters=filters, kernel_size=2, activation='relu', padding='same')(x)
    shape = K.int_shape(x)
    #x = Flatten()(x)
    enc = Dense(latent_dim)(x)
    
    encoder = Model(input, enc)
    if verbose:
        encoder.summary()
    
    latent_inputs = Input(shape=(latent_dim,), name='latent_input')
    #x = Dense(shape[1]*shape[2], name='Dense_after_enc', activation='relu')(latent_inputs)
    #x = Reshape((shape[1], shape[2]))(x)
    x = Dense(latent_dim)(latent_inputs)
    print(K.int_shape(x))
    for i in range(2):
        x = Conv1DTranspose(input_tensor=x, filters=filters, kernel_size=2, padding='same', last=False)
        #x = Dense(filters)(x)
        pass
    output = Dense(n_features, name='output')(x)
    
    decoder = Model(latent_inputs, output)
    if verbose:
        decoder.summary()
    
    output = decoder(encoder.output)
    vae = Model(input, output)
    
    vae.compile(optimizer='adam', loss='mse')
    
    return vae, encoder, decoder

vae, enc, dec = create_ae(verbose=True)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, None, 4)           0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, None, 50)          450       
_________________________________________________________________
conv1d_42 (Conv1D)           (None, None, 50)          5050      
_________________________________________________________________
dense_47 (Dense)             (None, None, 20)          1020      
Total params: 6,520
Trainable params: 6,520
Non-trainable params: 0
_________________________________________________________________
(None, 20)


ValueError: Input 0 is incompatible with layer conv2d_transpose_1: expected ndim=4, found ndim=3

In [186]:
def train():
    for batch in trainX_sequences:
        if len(batch) != 0:
            vae.fit(batch, batch, epochs=100, verbose=1)
        
print("TRAINING:")
train()
print("DONE")

TRAINING:


Epoch 1/100


32/54 [================>.............] - ETA: 1s - loss: 0.2942

54/54 [==============================] - 3s 47ms/step - loss: 0.2751


Epoch 2/100
32/54 [================>.............] - ETA: 0s - loss: 0.1886

...

68/68 [==============================] - 0s 4ms/step - loss: 2.1060e-05


Epoch 100/100
32/68 [=============>................] - ETA: 0s - loss: 2.1993e-05

64/68 [===========================>..] - ETA: 0s - loss: 1.9843e-05

68/68 [==============================] - 0s 5ms/step - loss: 2.0057e-05


DONE


In [187]:
from functools import reduce

from sklearn.decomposition import PCA

#masks = [return_mask(num, labels)[:, 0] for num in range(0, 9)]
encodings = []
for i,seqs in enumerate(trainX_sequences): 
    if len(seqs) != 0:
        encodings.append(np.mean(enc.predict(seqs), axis=1))
    
encodings_squeezed = []
print(len(encodings), encodings[0].shape)


5 (54, 20)


In [188]:

for i in encodings:
    for j in i:
        encodings_squeezed.append(j)
encs = np.array(encodings_squeezed)
print(encs.shape)


(300, 20)


In [189]:
def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg

    
masks = [return_mask(num, np.array(labels_ordered))[:, 0] for num in range(0, 9)]


In [190]:
from mpl_toolkits.mplot3d import Axes3D  


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask],
                    alpha=0.5)
        #break
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    

scaler = StandardScaler()
enc_input = scaler.fit_transform(encs) 
pca = PCA(3)
principalComponents = pca.fit_transform(enc_input)
print(principalComponents.shape)
print(pca.explained_variance_ratio_)
plot_pca('Sequences', 0)

principalComponents = encs
plot_pca('Sequences_Not_Pca', 0)



(300, 3)
[0.52323127 0.23315239 0.14782067]
